# ORM Integration
This notebook will look into integrating the Turing SDK by implementing the `ShortAnswerQuestion` object into an ORM model. For the purpose of simplicity, we will use the Django ORM in this notebook. However, as you will see, this integration works regardless of your ORM choice.

To grade questions with Turing, you need to implement the two stages of the question grading lifecycle:
1. Question object creation
2. Question grading request

Every request for question grading must address these two steps in one way or another. First, let's worry about creating a question object. As the name of this notebook suggests, we are going to create questions via an ORM model. To do this, we can implement an method on the ORM model, which utitlizes the fields of the model to create a `ShortAnswerQuestion` object.

First let's set the stage with an ORM model.

## Defining an ORM Model

In [ ]:
from django.db import models

class QuizQuestion(models.Model):
    FACTUAL = "FA"
    OPTION = "OP"
    ANALYSIS = "AN"
    QUESTION_TYPE_CHOICES = [
        (FACTUAL, "Factual"),
        (OPINION, "Opinion"),
        (ANALYSIS, "Analysis")
    ]

    _id = models.UUIDField(primary_key=True)
    question = models.CharField(max_length=100)
    example_answer = models.CharField(max_length=250)
    question_type = models.CharField(max_length=2, choices =QUESTION_TYPE_CHOICES, default='FACTUAL')

    def __str__(self):
        return self.question

Here we can find a fairly simple Django model for a Quiz Question. It defines a few basic fields for the database table and a standard `__str__` method, per the reccomendations of Django's docs. 

Now that we have our basic model setup, we can extend the model to easily create turing `ShortAnswerQuestion` objects directly through the model.

## Creating `ShortAnswerQuestion` Objects

In [ ]:
from django.db import models

from turing.question import ShortAnswerQuestion, RubricType

class QuizQuestion(models.Model):
    FACTUAL = "FA"
    OPTION = "OP"
    ANALYSIS = "AN"
    QUESTION_TYPE_CHOICES = [
        (FACTUAL, "Factual"),
        (OPINION, "Opinion"),
        (ANALYSIS, "Analysis")
    ]

    _id = models.UUIDField(primary_key=True)
    question = models.CharField(max_length=100)
    example_answer = models.CharField(max_length=250)
    question_type = models.CharField(max_length=2, choices =QUESTION_TYPE_CHOICES, default='FACTUAL')

    def __str__(self):
        return self.question

    @property
    def rubric_type(self):
        """Converts the question_type field to a RubricType object."""
        match self.question_type:
            case self.FACTUAL:
                return RubricType.FACTUAL
            case self.OPINION:
                return RubricType.COMMUNICATION_RUBRIC
            case self.ANALYSIS:
                return RubricType.ANALYTICAL_RUBRIC
            case _:
                return RubricType.CUSTOM_RUBRIC

    @property
    def turing_question(self):
        """Converts the QuizQuestion to a Turing ShortAnswerQuestion"""
        return ShortAnswerQuestion.from_rubric_type(
            question=self.question,
            example_answer=self.example_answer,
            rubric_type=self.rubric_type
        )

By implementing two simple properties into our ORM model, we were able to convert our standard Django model into a `ShortAnswerQuestion` factory. Now, we can simply use these properties to create and grade `ShortAnswerQuestion` objects anywhere in our app.

## Grading a Question
To keep with the Django theme, let's imagine we are in a view function now. We will call this the question grading view. In this view, we will query for a question through the `QuizQuestion` model, then use our properites and the provided `grade` method of the `ShortAnswerQuestion` object to grade the student's response with Turing. 

In [ ]:
from django.http import HttpRequest, HttpResponse
from django.views.decorators.http import require_http_methods

@require_http_methods(["POST"])
def grade_question_view(request: HttpRequest, question_id: str) -> HttpResponse:
    """Grades a student's response to a question."""

    # Get the question from the database
    question = QuizQuestion.objects.get(_id=question_id)

    # Use the custom property to get the turing question from the 
    # question model
    turing_question = question.turing_question

    # Get the student's answer from the request body
    answer = request.POST.get("answer")

    # Grade the question with turing
    feedback, score = turing_question.grade(answer)

    # Return the results to the client
    return HttpResponse({'feedback': feedback, 'score': score})


Just like that, we have extended our question model and implemented a view to access Turing's API. 